# Working with databases

This demo will show

- how to create a database called _Demo_
- how to add row based filegroup and file into it
- how to configure some properties of the database

## Creating new database
Following statement will create a database called *Demo*. Names and file paths should be adjusted with real environment if needed.

In [ ]:
use master
go

-- some house keeping
drop database if exists Demo
go

create database Demo
ON
(
    name = 'DemoData'
    , filename = 'F:\Data\Demo.mdf'
    , size = 256 MB
    , maxsize = 2 GB
    , filegrowth = 256 MB
)
LOG ON
(
    name = 'DemoLog'
    , filename = 'F:\Log\Demo.ldf'
    , size = 256 MB
    , maxsize = 2 GB
    , filegrowth = 256 MB
)
GO

Let's comment couple of interesting points in preceding statement:
- The simplest statement for a database creation is **CREATE DATABASE Demo**, but it leaves all file properties at default values, which's meaningless. 
- That's why each file is described separately.
    - each file contains logical name (the *NAME* property) used for administrative purposes
    - each file contains physical location (the *FILENAME* property)
    - size should be set of meaningful value regarding estimated amount of data
    - the *AUTOGROWTH* and *MAXSIZE* properties allows SQL Server to ask OS for more room in the file (increment is set by *AUTOGROWTH* property). This property is used for exceptional cases like very big transaction loading big amount of data. ***Manual growth is preferred***.

### Metadata of newly created database
- the database itself
- list of files
- list of filegroups

In [ ]:
use Demo
go

select database_id
    , file_id
    , type_desc
    , physical_name
    , state_desc
    , [size]
    , max_size
    , growth
from sys.master_files where database_id = db_id('Demo');

select name
    , type_desc
    , is_default
    , is_read_only
    , is_autogrow_all_files
from sys.filegroups;

select * from sys.dm_db_file_space_usage;

## Adding a filegroup to an existing database

Following example shows how to add a filegroup to our *Demo* db, and how to add file(s) to it.

In [8]:
use master
go

alter database Demo add filegroup FG1
go

alter database Demo add file
(
    name = 'DemoData2'
    , filename = 'F:\Data\Demo2.ndf'
    , size = 256 MB
    , maxsize = 2 GB
    , filegrowth = 256 MB
) to filegroup FG1

## Working with database properties
When we want to configure some of database properties, we will use three types of syntax
- **ALTER DATABASE SCOPED CONFIGURATION**
- **ALTER DATABASE name SET property = value** or **ALTER DATABASE name SET property value**
- **ALTER DATABASE name SET property**

As there is no visible rule which type of syntax is the right one, we can follow docs.microsoft.com documentation, or we can use SQL Server Management Studio. Anyway, some of examples are provided as follows

In [ ]:
-- let's set the database to single user mode
use master
go

alter database Demo set single_user with rollback immediate
go
-- let's allow the database to be accessible back again
alter database Demo set multi_user
go

-- let's configure proper recovery model of a database
alter database Demo set recovery full
go

-- let's limit the number of CPUs attending in one parallel query
use Demo
go
alter database scoped configuration set maxdop = 2
go